# Hoja de Trabajo 8
Máquinas Vectoriales de Soporte (SVM)

Michelle Mejía, Silvia Illescas y Emilio Reyes


1. Use los mismos conjuntos de entrenamiento y prueba de las hojas de trabajo pasadas para
probar el algoritmo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Cargar los datos
df = pd.read_csv("../dataset/train.csv")